In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import arviz as az
import numpy as np
import pymc as pm

def sampler_kwargs():
    return dict(
        nuts_sampler="nutpie",
        chains=2,
        draws=300,
        nuts_sampler_kwargs={"backend": "jax", "gradient_backend": "jax"},
    )

project_dir = Path.cwd().parent
train_df = pd.read_csv(project_dir / "data" / "model" / "training_data.csv",
    parse_dates=True,
    date_format="%Y %b",
    index_col=0,
    dtype={"value": float},
)

M = 30
D = 20

train_df['y'] = train_df.sum(axis=1)
train_df.index = pd.to_datetime(train_df.index)
train_df.index.freq = train_df.index.inferred_freq


In [3]:
N = len(train_df) - D + 1
reporting_matrices = np.empty((N, D, D))

def mask_matrix(matrix):
        masked_matrix = matrix.copy()
        for i in range(D):
            for j in range(D):
                if i + j > D - 1:
                     masked_matrix[i, j] = 0
        return  masked_matrix

for i in range(N):
    t = i + D
    delay_matrix = train_df.copy().drop('y', axis=1)
    delay_matrix = np.array(delay_matrix.iloc[t - D : t].values)
    masked_delay_matrix = mask_matrix(delay_matrix)

    reporting_matrices[i] = masked_delay_matrix

